In [2]:
!pip install -U datasets
!pip install -qU transformers accelerate bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("royboy0416/ko-alpaca")

# 데이터셋의 구조 확인
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/286 [00:00<?, ?B/s]

ko_alpaca_data.snappy.parquet:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 49620
    })
})


In [4]:
# gemma 데이터셋 포맷
# <start_of_turn>user
# What is Cramer's Rule?<end_of_turn>
# <start_of_turn>model
# Cramer's Rule is ...<end_of_turn>


# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""user\n{example["instruction"]}\n{example["input"]}\nmodel\n{example["output"]}"""
    # input 필드가 없는 경우
    else:
        text = f"""user\n{example["instruction"]}\nmodel\n{example["output"]}"""

    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = dataset.map(format_instruction)



Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'prompt': 'user\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\nmodel\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}

In [6]:
dataset['train'][5]

{'instruction': '홀수 중 하나를 밝히세요.',
 'input': '트위터, 인스타그램, 텔레그램',
 'output': '텔레그램입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n홀수 중 하나를 밝히세요.\n\n### Input:\n트위터, 인스타그램, 텔레그램\n\n### Response:\n텔레그램입니다.',
 'prompt': 'user\n홀수 중 하나를 밝히세요.\n트위터, 인스타그램, 텔레그램\nmodel\n텔레그램입니다.'}

### 모델 로드 및 튜닝:
1. 모델 학습: gemma-2b 모델을 로드하고, 준비된 데이터셋을 사용하여 모델을 세부 튜닝합니다. 이 과정에서는 학습률, 에폭 수 등의 파라미터를 조정할 수 있습니다.
2. 평가 및 반복: 튜닝된 모델을 평가하고 결과를 확인합니다. 필요에 따라 여러 번 반복하여 모델의 성능을 최적화할 수 있습니다.

In [7]:
!nvidia-smi

Fri Nov  8 00:14:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
notebook_login()

In [11]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 39696
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 9924
    })
})

In [14]:
train_data = dataset["train"]
test_data = dataset["test"]

In [15]:
print(train_data[0])

{'instruction': '희망의 감정을 전달하는 노래를 작곡하세요.', 'input': '', 'output': '저의 노래는 악기적인 연주와 함께 멜로디를 통해 희망의 감정을 전달합니다.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n희망의 감정을 전달하는 노래를 작곡하세요.\n\n### Response:\n저의 노래는 악기적인 연주와 함께 멜로디를 통해 희망의 감정을 전달합니다.', 'prompt': 'user\n희망의 감정을 전달하는 노래를 작곡하세요.\nmodel\n저의 노래는 악기적인 연주와 함께 멜로디를 통해 희망의 감정을 전달합니다.', 'input_ids': [2, 1645, 108, 240498, 240547, 236137, 99091, 236864, 236392, 31087, 239522, 40284, 61138, 238608, 236791, 63806, 241017, 204551, 235265, 108, 2516, 108, 238721, 236137, 61138, 238608, 236214, 223671, 236386, 85024, 50833, 237014, 237807, 135132, 235248, 244521, 236375, 238173, 236791, 187118, 235248, 240498, 240547, 236137, 99091, 236864, 236392, 31087, 239522, 43395, 235265, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """user
  {query}

  model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)

user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  


In [17]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due 

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,2.773600
20,2.354100
30,2.092500
40,1.992300
50,1.955400
60,1.925500
70,1.879400
80,1.754300
90,1.837700
100,1.859000


TrainOutput(global_step=100, training_loss=2.0423722076416015, metrics={'train_runtime': 221.6161, 'train_samples_per_second': 1.805, 'train_steps_per_second': 0.451, 'total_flos': 406145636941824.0, 'train_loss': 2.0423722076416015, 'epoch': 0.01007658202337767})

In [18]:
# Fine tuning 이후
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)


user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
   1. 규칙적으로 운동을 하세요.
  2. 건강한 식사를 즐겨 먹으세요.
  3. 규칙적으로 걷기와 걷기 운동을 하세요.
  
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  건강한 식사를 즐겨 먹으세요.
  model
  


In [19]:
# Fine tuning 이후
result = get_completion(query="불면증을 해결하는 방법을 세 가지 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  불면증을 해결하는 방법을 세 가지 알려주세요.
  
  model
   1. 잠을 깨운다.
  2. 잠을 깨운다.
  3. 잠을 깨운다.
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  model
  


### 모델 저

In [20]:
new_model = "gemma-2b-it-koalpaca-peft"
trainer.model.save_pretrained(new_model)